In [4]:
import pandas as pd
import numpy as np

import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State

import plotly.graph_objects as go

import os
df_input_large=pd.read_csv('../../data/processed/COVID_final_set.csv',sep=';')
print(df_input_large.head(5))

         date    state     country  confirmed  confirmed_filtered  \
0  2020-01-22  Alberta      Canada        0.0                 0.0   
1  2020-01-22       no      Kosovo        0.0                 0.0   
2  2020-01-22       no      Kuwait        0.0                 0.0   
3  2020-01-22       no  Kyrgyzstan        0.0                 0.0   
4  2020-01-22       no        Laos        0.0                 0.0   

   confirmed_DR  confirmed_filtered_DR  
0           NaN                    NaN  
1           NaN                    NaN  
2           NaN                    NaN  
3           NaN                    NaN  
4           NaN                    NaN  


In [5]:
fig = go.Figure()

app = dash.Dash()
app.layout = html.Div([
    dcc.Markdown('''
    #  Applied Data Science on COVID-19 data
    Goal of the project is to teach data science by applying a cross industry standard process,
    it covers the full walkthrough of: automated data gathering, data transformations,
    filtering and machine learning to approximating the doubling time, and
    (static) deployment of responsive dashboard.
    '''),
    
    dcc.Markdown('''
    ## Multi-Select Country for visualization
    '''),

    dcc.Dropdown(
        id='country_drop_down',
        options=[ {'label': country,'value':country} for country in df_input_large['country'].unique()],
        value=['US', 'Germany','India'], # pre-selected
        multi=True
    ),
    
    dcc.Markdown('''
        ## Select Timeline of confirmed COVID-19 cases or the approximated doubling time
    '''),
    
    dcc.Dropdown(
    id='doubling_time',
    options=[
        {'label': 'Timeline Confirmed ', 'value': 'confirmed'},
        {'label': 'Timeline Confirmed Filtered', 'value': 'confirmed_filtered'},
        {'label': 'Timeline Doubling Rate', 'value': 'confirmed_DR'},
        {'label': 'Timeline Doubling Rate Filtered', 'value': 'confirmed_filtered_DR'},
    ],
    value='confirmed',
    multi=False
    ),
    
    dcc.Graph(figure=fig, id='main_window_slope')
])


@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value'),
    Input('doubling_time', 'value')])
def update_figure(country_list,show_doubling):
    if 'doubling_rate' in show_doubling:
        my_yaxis={
            'type':"log",
            'title':'Approximated doubling rate over 3 days (larger numbers are better #stayathome)'
              }
    else:
        my_yaxis={
            'type':"log",
            'title':'Confirmed infected people (source johns hopkins csse, log-scale)'
            }
        
    traces = []
    for country in country_list:
        df_plot=df_input_large[df_input_large['country']==country]
        if show_doubling=='doubling_rate_filtered':
            df_plot=df_plot[['state','country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['country','date']).agg(np.mean).reset_index()
        else:
            df_plot=df_plot[['state','country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['country','date']).agg(np.sum).reset_index()
        
        traces.append(
            dict(
                    x=df_plot.date,
                    y=df_plot[show_doubling],
                    mode='markers+lines',
                    opacity=0.9,
                    name=country
                )
        )
    
    return {
            'data': traces,
            'layout': dict (
                            width=1280,
                            height=720,
                            xaxis={'title':'Timeline',
                                'tickangle':-45,
                                'nticks':20,
                                'tickfont':dict(size=14,color="#7f7f7f"),
                              },
                            yaxis=my_yaxis
                    )
            }

In [6]:
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
